In [ ]:
#after running this, no need to restart the kurnel when you modify other files.
%load_ext autoreload
%autoreload 2

In [20]:
import os
import math
import open3d
import csv
from settings import DATA_PATH
import clusterization.clusterize as clusterizer
import numpy as np
from utils.visualizer import save_camera_params
from utils.handling_cluster import visualize_by_cluster_bbox, crop_cluster, visualize_bbox_result
from detection.detect import compute_nearest_cluster_multicase, compute_bbox, save_bbox_result, change_cluster_color
import utils.visualizer as visualizer
from plane_elimination.plane_elimination import eliminate_planes
from plane_elimination.plane_equotions_list import planes_list, bool_list #list to define plane equotions

data_type_name_pcd = "LIVOX_Hallway_pcds"
experiment_name = "jogging_fast_4th"
dir_name = "res100ms_start13.5s"

#directory path of pcd files
directory_path = os.path.join(DATA_PATH, data_type_name_pcd, experiment_name, dir_name)
#'{DATA_PATH}/LIVOX_Hallway_pcds/jogging_fast_4th/res100ms_start13.5s'

#add save_path
experiment_name_clustered = experiment_name + "_clustered"
dir_name_clustered= dir_name + "_clustered"
folder_clustered = os.path.join(DATA_PATH, data_type_name_pcd, experiment_name_clustered, dir_name_clustered)

if not os.path.exists(folder_clustered):
    os.makedirs(folder_clustered)

# make the list of files
file_list = []
for filename in os.listdir(directory_path):
    if os.path.isfile(os.path.join(directory_path, filename)):
        file_list.append(os.path.join(directory_path, filename))
        
#select your view point and close your window.
pcd = open3d.io.read_point_cloud(file_list[0])
cam_params = save_camera_params(pcd)

set the view point and close the window


In [ ]:
#add save_path
experiment_name_tracked = experiment_name + "_bbox_tracked"  #"jogging_fast_4th_bbox_tracked"
dir_name_tracked= dir_name + "_bbox_tracked"  #res100ms_start13.5s_bbox_tracked
folder_tracked = os.path.join(DATA_PATH, data_type_name_pcd, experiment_name_tracked, dir_name_tracked)
#.\data\LIVOX_Hallway_pcds\jogging_fast_4th_bbox_tracked\res100ms_start13.5s_bbox_tracked

if not os.path.exists(folder_tracked):
    os.makedirs(folder_tracked)

#initialization
eps = 0.5
center_prev_list = []  #initialize the center list for the next frame
center_prev = np.zeros(3) #initialize center of human cluster in previous frame
cluster_ids = []

#clusterize each files and visualize.
for i, pcd_file in enumerate(file_list):
    pcd = open3d.io.read_point_cloud(pcd_file)
    
    pcd_clustered = folder_clustered + "/" + os.path.basename(pcd_file) #pcd_clustered_save_path   
    
    #eliminate points by planes
    pcd = eliminate_planes(pcd, planes_list=planes_list, bool_list=bool_list, visualize=False)
    
    #clusterization
    pcd, labels = clusterizer.clusterize(pcd, eps=eps,
                             time_duration=None, #time duration that each windows open for. If you choose None, close window manually.
                             window_name=os.path.basename(pcd_file), #window name changes accorrding to file name.
                             cam_params=cam_params,#camera parameter(view point) you selected
                             save_path=pcd_clustered,
                             visualize=False
                             ) 
    
    
    filtered_labels = compute_bbox(pcd,labels)
    
    
    #color of points in cluster_id is changed into red.
    #pcd = change_cluster_color(pcd, labels, cluster_id=filtered_labels)
    #visualizer.visualize(pcd, time_duration=None, cam_params=cam_params)
    
    #select human cluster in first frame
    if i==0:
        visualize_by_cluster_bbox(pcd, labels, filtered_labels)
        filtered_labels = input("Please enter cluster ids, format example: 0,1,2.. ").split(',')
        filtered_labels = [int(x) for x in filtered_labels]
        
    human_clusters = np.empty((0,3))
    #select cluster id in comand line. check each window name.
    for inx, fl_label in enumerate(filtered_labels):
        if i==0:                                     # for the first frame
            cluster_id = fl_label
        else:
            inx, cluster_id = compute_nearest_cluster_multicase(pcd, labels, fl_label, center_prev_list)
            
        #crop (and save) selected cluser
        cropped_pcd = crop_cluster(pcd, labels, cluster_id=cluster_id, visualize=False)
        
        center_tmp = cropped_pcd.get_center()

        
        if i==0:   
            center_prev_list.append([inx, center_tmp])

            # save the cropped cluster(pcd, save_path)
            save_bbox_result(inx, folder_tracked, pcd_file, cropped_pcd)

            human_clusters = np.append(human_clusters, cropped_pcd.points, axis=0)
            
        else:  #update center if current cluster is close to previous one
            center_prev = center_prev_list[inx][1]
            current_dist = np.linalg.norm (center_prev - center_tmp)
            if 0.1< current_dist < 5:
                #update the previous center list
                center_prev_list[inx][1] = center_tmp
                # save the cropped cluster(pcd, save_path)
                save_bbox_result(inx, folder_tracked, pcd_file, cropped_pcd)
                
                human_clusters = np.append(human_clusters, cropped_pcd.points, axis=0)

    # visulize the bbox result of each frame
    visualize_bbox_result(pcd, human_clusters, time_duration=0.2, cam_params=cam_params, visualize=True)


In [21]:
#### get center of mass of the cluster and calculate velocity ###

# get each subfolders in folder_tracked (different players if exist)
subfolders = [f.path for f in os.scandir(folder_tracked) if f.is_dir()]

for subfolder in subfolders:
    print(f"Processing folder: {subfolder}")
    subfolder_name = os.path.basename(subfolder)
    
    center_list = []
    time_list = []
    pcd_files = [f.path for f in os.scandir(subfolder) if f.is_file() and f.name.endswith(".pcd")]
    
    # Extracting file names without the ".pcd" extension
    pcd_names_without_extension = [os.path.splitext(os.path.basename(pcd_file))[0] for pcd_file in pcd_files]

    for pcd_file in pcd_files:
        print(f"Reading PCD file: {pcd_file}")
        
        # file name without extension (frame time)
        frame_time = os.path.splitext(os.path.basename(pcd_file))[0]
        
        # read point cloud cluster
        point_cloud = open3d.io.read_point_cloud(pcd_file)
        
        # get center of cluster
        center = point_cloud.get_center()
        
        center_list.append(center) 
        time_list.append(frame_time)  # unit: ms

    
    # calculate velocity
    # take only the x,y-axis
    centers_xy = [row[:2] for row in center_list]
    time_list = [float(num) for num in time_list]
    
    velocities = []
    csv_data = []
    
    for i in range(1, len(centers_xy)):
        # calculate displacement
        displacement_x = centers_xy[i][0] - centers_xy[i - 1][0]
        displacement_y = centers_xy[i][1] - centers_xy[i - 1][1]

        # calculate time interval
        time_interval = time_list[i] - time_list[i - 1]

        velocity_x = 1000 * displacement_x / time_interval  # *1000 cuz of the time unit is ms
        velocity_y = 1000 * displacement_y / time_interval

        velocity = math.sqrt(velocity_x**2 + velocity_y**2)
        
        velocities.append(velocity)
        
        csv_data.append((time_list[i], velocity))


    # save velocity data to a csv file
    csv_file = "velocity.csv"
    csv_path = os.path.join(folder_tracked, subfolder_name, csv_file)
    
    with open(csv_path, mode='w', newline='') as file:
        writer = csv.writer(file)

        writer.writerow(['Time', 'Velocity'])

        for time, velocity in csv_data:
            writer.writerow([time, velocity])


print("end")



Processing folder: D:/TUM File/semester5/pratical laser scanning/data\LIVOX_Hallway_pcds\jogging_fast_4th_bbox_tracked\res100ms_start13.5s_bbox_tracked\0_class
Reading PCD file: D:/TUM File/semester5/pratical laser scanning/data\LIVOX_Hallway_pcds\jogging_fast_4th_bbox_tracked\res100ms_start13.5s_bbox_tracked\0_class\00000.pcd
Reading PCD file: D:/TUM File/semester5/pratical laser scanning/data\LIVOX_Hallway_pcds\jogging_fast_4th_bbox_tracked\res100ms_start13.5s_bbox_tracked\0_class\00100.pcd
Reading PCD file: D:/TUM File/semester5/pratical laser scanning/data\LIVOX_Hallway_pcds\jogging_fast_4th_bbox_tracked\res100ms_start13.5s_bbox_tracked\0_class\00200.pcd
Reading PCD file: D:/TUM File/semester5/pratical laser scanning/data\LIVOX_Hallway_pcds\jogging_fast_4th_bbox_tracked\res100ms_start13.5s_bbox_tracked\0_class\00300.pcd
Reading PCD file: D:/TUM File/semester5/pratical laser scanning/data\LIVOX_Hallway_pcds\jogging_fast_4th_bbox_tracked\res100ms_start13.5s_bbox_tracked\0_class\00400